In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import pandas as pd

[nltk_data] Downloading package wordnet to /Users/vignesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
d1 = pd.read_csv("NewsMar29.csv",)

In [4]:
headlinesdf=d1[['Headline']]

In [5]:
headlinesdf['index'] = headlinesdf.index

In [6]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [7]:
doc_sample = headlinesdf[headlinesdf['index'] == 52].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Trump', 'asked', 'pharmaceutical', 'execs', 'if', 'the', 'flu', 'vaccine', 'could', 'be', 'used', 'to', 'stop', 'coronavirus.', "Here's", 'why', 'that', "wouldn't", 'work.']


 tokenized and lemmatized document: 
['trump', 'ask', 'pharmaceut', 'exec', 'vaccin', 'stop', 'coronavirus', 'wouldn', 'work']


In [8]:
processed_docs = headlinesdf['Headline'].map(preprocess)
processed_docs[:10]

0    [appl, reject, coronavirus, app, limit, virus,...
1                [bogus, coronavirus, tracker, infect]
2    [trump, cross, corona, replac, chines, press, ...
3          [disney, tangl, coronavirus, googl, search]
4                                [corona, cancel, con]
5         [global, pandem, stop, desper, desir, viral]
6                                    [tip, wash, hand]
7    [photograph, captur, trump, handwritten, chine...
8     [japan, hotel, lose, olymp, coronavirus, spread]
9    [startup, felix, want, replac, antibiot, progr...
Name: Headline, dtype: object

In [9]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 app
1 appl
2 coronavirus
3 limit
4 misinform
5 reject
6 virus
7 bogus
8 infect
9 tracker
10 chines


In [10]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[50]

[(1, 1), (3, 1)]

In [12]:
bow_doc_50 = bow_corpus[50]
for i in range(len(bow_doc_50)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_50[i][0], 
                                               dictionary[bow_doc_50[i][0]], 
bow_doc_50[i][1]))

Word 1 ("virus") appears 1 time.
Word 3 ("trump") appears 1 time.


In [13]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.44158362601351453), (1, 0.8972200963179305)]


In [14]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.436*"trump" + 0.309*"corona" + 0.191*"virus" + 0.037*"coronavirus" + 0.009*"covid" + 0.009*"spread" + 0.009*"world"
Topic: 1 
Words: 0.876*"coronavirus" + 0.067*"world" + 0.044*"spread" + 0.003*"corona" + 0.003*"virus" + 0.003*"covid" + 0.003*"trump"
Topic: 2 
Words: 0.446*"spread" + 0.404*"coronavirus" + 0.043*"world" + 0.043*"covid" + 0.043*"trump" + 0.011*"virus" + 0.011*"corona"
Topic: 3 
Words: 0.525*"coronavirus" + 0.256*"trump" + 0.134*"spread" + 0.049*"virus" + 0.012*"corona" + 0.012*"covid" + 0.012*"world"
Topic: 4 
Words: 0.659*"covid" + 0.178*"coronavirus" + 0.133*"spread" + 0.007*"corona" + 0.007*"virus" + 0.007*"trump" + 0.007*"world"
Topic: 5 
Words: 0.464*"corona" + 0.232*"coronavirus" + 0.179*"virus" + 0.071*"spread" + 0.018*"covid" + 0.018*"trump" + 0.018*"world"
Topic: 6 
Words: 0.683*"corona" + 0.225*"covid" + 0.033*"virus" + 0.033*"spread" + 0.008*"coronavirus" + 0.008*"world" + 0.008*"trump"
Topic: 7 
Words: 0.343*"world" + 0.219*"virus" + 0.125*

In [16]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=10, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.751*"world" + 0.109*"virus" + 0.072*"coronavirus" + 0.017*"corona" + 0.017*"spread" + 0.017*"covid" + 0.017*"trump"
Topic: 1 Word: 0.163*"coronavirus" + 0.139*"corona" + 0.139*"virus" + 0.139*"covid" + 0.139*"spread" + 0.139*"world" + 0.139*"trump"
Topic: 2 Word: 0.982*"coronavirus" + 0.003*"corona" + 0.003*"virus" + 0.003*"covid" + 0.003*"spread" + 0.003*"trump" + 0.003*"world"
Topic: 3 Word: 0.749*"spread" + 0.201*"coronavirus" + 0.010*"covid" + 0.010*"world" + 0.010*"trump" + 0.010*"virus" + 0.010*"corona"
Topic: 4 Word: 0.863*"trump" + 0.047*"coronavirus" + 0.024*"corona" + 0.021*"spread" + 0.015*"world" + 0.015*"covid" + 0.015*"virus"
Topic: 5 Word: 0.145*"coronavirus" + 0.145*"virus" + 0.145*"covid" + 0.144*"trump" + 0.142*"corona" + 0.139*"spread" + 0.139*"world"
Topic: 6 Word: 0.728*"corona" + 0.246*"virus" + 0.005*"spread" + 0.005*"covid" + 0.005*"coronavirus" + 0.005*"trump" + 0.005*"world"
Topic: 7 Word: 0.184*"coronavirus" + 0.145*"corona" + 0.134*"virus" +